In [5]:
from torchvision import utils
from basic_fcn import *
from dataloader import *
from utils import *
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import time
import math
from tqdm import tqdm
import gc

In [6]:
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform_(m.weight.data)
        torch.nn.init.xavier_uniform_(m.bias.data.view(m.bias.data.shape[0],1))
        #a = math.sqrt(3) * math.sqrt(2/m.bias.data.shape[0])
        #torch.nn.init._no_grad_uniform_(m.bias.data, -a, a)
        
        


In [7]:

    
def train(model, criterion, epochs, train_loader, val_loader, test_loader, use_gpu, name):

    optimizer = optim.Adam(fcn_model.parameters(), lr=5e-3)
    if use_gpu:
        device = torch.device("cuda:0")
        model = torch.nn.DataParallel(model)
        model.to(device)
        
        
    
    val_loss_set = []
    val_acc_set = []
    val_iou_set = []
    
    # Early Stop criteria
    minLoss = 1e6
    minLossIdx = 0
    earliestStopEpoch = 10
    earlyStopDelta = 5
    for epoch in range(epochs):
        ts = time.time()
        
        #print(np.array(val_loss).shape)
        # early-stopping 
#         if epoch > 11:
#             if val_loss[-1] < val_loss[-10]:
#                 open('save_param', 'w').close()
#                 torch.save(fcn_model.state_dict(), 'save_param')
                
                  
        for iter, (inputs, tar, labels) in tqdm(enumerate(train_loader)):
            #print("\n**********************************************\nIter")
            #checkM()
            optimizer.zero_grad()
            del tar
            
            if use_gpu:
                inputs = inputs.to(device)# Move your inputs onto the gpu
                labels = labels.to(device) # Move your labels onto the gpu
            
                
            outputs = model(inputs)
            del inputs
            loss = criterion(outputs, Variable(labels.long()))
            del labels
            del outputs
            #print("\n**********************************************\nPre back")
            #checkM()
            loss.backward()
            loss = loss.item()
            optimizer.step()

            if iter % 10 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss))
            
            #print("\n^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n")
        
        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        
        # calculate val loss each epoch
        val_loss, val_acc, val_iou = val(model, val_loader, criterion, use_gpu)
        val_loss_set.append(val_loss.item())
        val_acc_set.append(val_acc.item())
        val_iou_set.append(val_iou.item())
        
        # Early stopping
        if val_loss < minLoss:
            # Store new best
            torch.save(model, name)
            minLoss = val_loss.item()
            minLossIdx = epoch
            
        # If passed min threshold, and no new min has been reached for delta epochs
        elif epoch > earliestStopEpoch and (epoch - minLossIdx) > earlyStopDelta:
            print("Stopping early at {}".format(minLossIdx))
            break
        # TODO what is this for?
        #model.train()
        
    return val_loss_set, val_acc_set, val_iou_set, predictions


def val(model, val_loader, criterion, use_gpu):
    
    # set to evaluation mode 
    model.eval()

    softmax = nn.Softmax(dim = 1)
    
    loss = []
    pred = []
    acc = []
    
    IOU_init = False
    if use_gpu:
        device = torch.device("cuda:0")
        
        #model.to(device)
        
    for iter, (X, tar, Y) in tqdm(enumerate(val_loader)):
        
        if not IOU_init:
            IOU_init = True
            IOU = np.zeros((1,tar.shape[1]))
            
        if use_gpu:
            inputs = X.to(device)
            labels = Y.to(device)
            
        else:
            inputs, labels = X, Y

            
        with torch.no_grad():   
            outputs = model(inputs)    
            loss.append(criterion(outputs, labels.long()).item())
            prediction = softmax(outputs) 
            acc.append(pixel_acc(prediction, labels).item())
            IOU = IOU + np.array(iou(prediction, labels))
        
    
    acc = sum(acc)/len(acc)
    avg_loss = sum(loss)/len(loss) 
    IOU = IOU/iter  
    
    return avg_loss, acc, IOU      
       
    
    
    
def test(model, use_gpu):
    
    softmax = nn.Softmax(dim = 1)
    
    pred = []
    acc = []
    if use_gpu:
        device = torch.device("cuda:0")
        
        model.to(device)
    
    IOU_init = False
    for iter, (X, tar, Y) in enumerate(test_loader):
        
        if not IOU_init:
            IOU_init = True
            IOU = np.zeros((1,tar.shape[1]))
        
        if use_gpu:
            inputs = X.to(device)
            labels = Y.to(device)
        else:
            inputs, labels = X, Y
                    
        
        outputs = fcn_model(inputs)  
        
        prediction = softmax(outputs)
        acc.append(pixel_acc(prediction, labels))
        IOU = IOU + np.array(iou(prediction, Y))
        
    acc = sum(acc)/len(acc)        
    IOU = IOU/iter

    #Complete this function - Calculate accuracy and IoU 
    # Make sure to include a softmax after the output from your model
    
    return acc, IOU
    


In [8]:
def checkM():
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
                print(type(obj), obj.size())
        except:
            pass

if __name__ == "__main__":
    train_dataset = CityScapesDataset(csv_file='train.csv')
    val_dataset = CityScapesDataset(csv_file='val.csv')
    test_dataset = CityScapesDataset(csv_file='test.csv')
    train_loader = DataLoader(dataset=train_dataset,
                          batch_size=6,
                          num_workers=8,
                          shuffle=True)
    val_loader = DataLoader(dataset=val_dataset,
                          batch_size=8,
                          num_workers=8,
                          shuffle=True)
    test_loader = DataLoader(dataset=test_dataset,
                          batch_size=4,
                          num_workers=4,
                          shuffle=True)
    
    
    epochs     = 100
    criterion = torch.nn.CrossEntropyLoss()
    # Fix magic number
    fcn_model = FCN(n_class=34)
    fcn_model.apply(init_weights)
    
    
    epochs     = 100
    use_gpu = torch.cuda.is_available()
#     if use_gpu:
#         device = torch.device("cuda:0")
#         fcn_model = torch.nn.DataParallel(fcn_model)
#         fcn_model.to(device)
#     val(fcn_model, val_loader, criterion, use_gpu)
    train(fcn_model, criterion, epochs, train_loader, val_loader, test_loader, use_gpu, "FCN")
    
    
    fcn_model.load_state_dict(torch.load('./save_param'))
    


0it [00:00, ?it/s]


**********************************************
Iter
<class 'torch.Tensor'> torch.Size([6, 3, 1024, 2048])
<class 'torch.Tensor'> torch.Size([6, 34, 1024, 2048])
<class 'torch.Tensor'> torch.Size([6, 1024, 2048])
<class 'torch.Tensor'> torch.Size([6, 3, 1024, 2048])
<class 'torch.Tensor'> torch.Size([6, 34, 1024, 2048])
<class 'torch.Tensor'> torch.Size([6, 1024, 2048])
<class 'torch.Tensor'> torch.Size([6, 3, 1024, 2048])
<class 'torch.Tensor'> torch.Size([6, 34, 1024, 2048])
<class 'torch.Tensor'> torch.Size([6, 1024, 2048])
<class 'torch.nn.parameter.Parameter'> torch.Size([512, 256, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([256])
<class 'torch.nn.parameter.Parameter'> torch.Size([256])
<class 'torch.nn.parameter.Parameter'> torch.Size([256])
<class 'torch.nn.parameter.Parameter'> torch.Size([512, 512, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([512])
<class 'torch.nn.parameter.Parameter'> torch.Size([512])
<class 'torch.nn.parameter.Parameter'> torch.S

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: RangeIndex.data is deprecated and will be removed in a future version
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Index.data is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


RuntimeError: CUDA out of memory. Tried to allocate 1.59 GiB (GPU 0; 10.92 GiB total capacity; 9.04 GiB already allocated; 255.50 MiB free; 1.11 GiB cached) (malloc at /opt/conda/conda-bld/pytorch_1573049310284/work/c10/cuda/CUDACachingAllocator.cpp:267)
frame #0: c10::Error::Error(c10::SourceLocation, std::string const&) + 0x47 (0x7f6c5e78f687 in /opt/conda/lib/python3.7/site-packages/torch/lib/libc10.so)
frame #1: <unknown function> + 0x1ea29 (0x7f6c5e9d2a29 in /opt/conda/lib/python3.7/site-packages/torch/lib/libc10_cuda.so)
frame #2: <unknown function> + 0x1fade (0x7f6c5e9d3ade in /opt/conda/lib/python3.7/site-packages/torch/lib/libc10_cuda.so)
frame #3: at::native::empty_cuda(c10::ArrayRef<long>, c10::TensorOptions const&, c10::optional<c10::MemoryFormat>) + 0x3f4 (0x7f6c6aa75854 in /opt/conda/lib/python3.7/site-packages/torch/lib/libtorch.so)
frame #4: <unknown function> + 0x3d94528 (0x7f6c69013528 in /opt/conda/lib/python3.7/site-packages/torch/lib/libtorch.so)
frame #5: <unknown